<center><h1>XGBRegressor, Optuna and K-Fold CV</h1></center>
<center><h2>Tabular Playground Series - March 2022</h2></center>
<center><h2>By Tariq Hussain</h2></center>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Imports

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

<h2>Data optimization and importing</h2>

In [ ]:
def reduce_mem_usage(df):
    #iterate through all the columns of a dataframe and modify the data type to reduce memory usage.
    
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(file):
    #create a dataframe and optimize its memory usage
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [ ]:
#train_fp = "../input/tabular-playground-series-mar-2022/train.csv"
#train_df = pd.read_csv(train_fp)

#test_fp = "../input/tabular-playground-series-mar-2022/test.csv"
#test_df = pd.read_csv(test_fp)

In [ ]:
train_df = import_data('../input/tabular-playground-series-mar-2022/train.csv')
test_df = import_data('../input/tabular-playground-series-mar-2022/test.csv')

# Data preparation

In [ ]:
train_df.head()

In [ ]:
train_df.info()

In [ ]:
test_df.head()

In [ ]:
test_df.info()

In [ ]:
train_copy = train_df.copy()
test_copy = test_df.copy()

In [ ]:
def convert_dates(df):
    
    df['date'] = pd.to_datetime(df['time'])
    
    #df['day'] = df.date.dt.day
    df['month'] = df.date.dt.month
    df['hour'] = df.date.dt.hour
    df['minute'] = df.date.dt.minute
    df['day_of_week'] = df.date.dt.weekday
    df['is_weekend'] = (df.date.dt.dayofweek > 4).astype(int)
    df['is_morning'] = (6 <= df['hour']) & (df['hour'] < 12)
    df['is_afternoon'] = (12 <= df['hour']) & (df['hour'] < 18)
    df['is_evening'] = (18 <= df['hour']) & (df['hour'] <= 23)
    df['is_night'] = (0 <= df['hour']) & (df['hour'] < 6)
    
    df['is_morning'] = df['is_morning'].astype(int)
    df['is_afternoon'] = df['is_afternoon'].astype(int)
    df['is_evening'] = df['is_evening'].astype(int)
    df['is_night'] = df['is_night'].astype(int)
    
    return df

train = convert_dates(train_copy)
test = convert_dates(test_copy)

In [ ]:
train

In [ ]:
train.info()

In [ ]:
test

In [ ]:
'''
train = pd.get_dummies(train, columns=["direction", "is_morning", "is_afternoon", "is_evening", "is_night"])
test  = pd.get_dummies(test, columns=["direction", "is_morning", "is_afternoon", "is_evening", "is_night"])

test.insert(18, column='is_morning_True', value=0)
test.insert(24, column='is_night_True', value=0)
'''

In [ ]:
X = train.drop(columns=["row_id", "congestion", "time", "date"])
y = train.congestion

test2 = test.drop(columns=["row_id", "time", "date"])

In [ ]:
test2.columns

In [ ]:
X

In [ ]:
test2.isnull().sum()

In [ ]:
test2

In [ ]:
#X_direction = X.direction

#test_direction = test.direction

In [ ]:
#X_direction

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

'''
cat_cols = [
    'direction',
    'is_morning',
    'is_afternoon',
    'is_evening',
    'is_night'
]
'''

le = LabelEncoder()

X_encoded = pd.DataFrame(le.fit_transform(X['direction']))#.toarray())
test_encoded = pd.DataFrame(le.fit_transform(test2['direction']))#.toarray())

X = X.join(X_encoded)
test2 = test2.join(test_encoded)

#pd.DataFrame(enc.fit_transform(bridge_df[['Bridge_Types_Cat']]).toarray())

In [ ]:
scaler = StandardScaler()

num_cols = [
    'x',
    'y',
    'month',
    'hour',
    'minute',
    'day_of_week'
]

X[num_cols] = scaler.fit_transform(X[num_cols])
test2[num_cols] = scaler.transform(test2[num_cols])

In [ ]:
X = X.drop(columns=['direction'], axis=0)
test2 = test2.drop(columns=['direction'], axis=0)

In [ ]:
#print(X_encoded)

In [ ]:
#test_encode

In [ ]:
#X['direction'] = X_encode

#test2['direction'] = test_encode

In [ ]:
X

In [ ]:
test2

# Modelling

In [ ]:
from sklearn.model_selection import StratifiedKFold, train_test_split, cross_val_score

xtrain,xvalid,ytrain,yvalid = train_test_split(X,y,test_size=0.2,random_state=0)

<h2>Hyperperameter optimization</h2>

In [ ]:
import optuna
from optuna.samplers import TPESampler
from hyperopt import STATUS_OK,Trials,fmin,hp,tpe
from xgboost import XGBClassifier, XGBRegressor
from sklearn.metrics import log_loss, accuracy_score, mean_absolute_error, r2_score, roc_auc_score
from optuna.integration import XGBoostPruningCallback

def smape(actual, predicted):
    numerator = np.abs(predicted - actual)
    denominator = (np.abs(actual) + np.abs(predicted)) / 2
    
    return np.mean(numerator / denominator)*100

def objective(trial):
    
    params = {
        "objective": trial.suggest_categorical('objective', ["reg:squarederror"]),
        "eval_metric": trial.suggest_categorical('eval_metric', ["mae"]),
        "use_label_encoder": trial.suggest_categorical('use_label_encoder', [False]),
        "n_estimators": trial.suggest_categorical('n_estimators', [40000]),
        "learning_rate": trial.suggest_loguniform('learning_rate', 0.15, 1.0),
        "subsample": trial.suggest_float('subsample', 0.1, 1, step=0.01),
        "colsample_bytree": trial.suggest_float('colsample_bytree', 0.05, 1, step=0.01),
        "max_depth": trial.suggest_int("max_depth", 1, 8),
        "booster": trial.suggest_categorical('booster', ["gbtree"]),
        "gamma": trial.suggest_float('gamma', 0, 100, step=0.1),
        "tree_method": trial.suggest_categorical('tree_method', ["gpu_hist"]),
        "reg_lambda": trial.suggest_loguniform('reg_lambda', 0.1, 100),
        "reg_alpha": trial.suggest_loguniform('reg_alpha', 0.1, 100),
        "random_state": trial.suggest_categorical('random_state', [42]),
        "n_jobs": trial.suggest_categorical('n_jobs', [4]),
        "min_child_weight": trial.suggest_categorical("min_child_weight", [256]),
        "gpu_id":0,
            }
    
    #opt_params = params
    #opt_params['n_estimators'] = 80000
    
    model = XGBRegressor(**params)

    model.fit(
        xtrain, 
        ytrain,
        early_stopping_rounds=100,
        eval_set=[(xvalid, yvalid)],
        #eval_metric='auc',
        verbose=False
    )

    yhat = model.predict(xvalid)
    #preds = model.predict(np.float32(xvalid))
    #yhat = preds#[:, 1]
    return smape(yvalid, yhat)
    #return mean_absolute_error(yvalid, yhat)

study = optuna.create_study(direction="minimize", sampler=TPESampler())
study.optimize(objective, n_trials=100)
#print(study.best_params)

In [ ]:
print("Best value: {:.5f}".format(study.best_value))
print("Best params:")

for key, value in study.best_params.items():
    print("{}: {}".format(key, value))

In [ ]:
xgb_params = study.best_params
xgb_params

In [ ]:
#test2.insert(20, column='14', value=0.0)
#test2.insert(21, column='15', value=0.0)

# K-Fold Cross Validation

In [ ]:

from sklearn.model_selection import TimeSeriesSplit

#kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

#preds = []
#scores = []

#%%time
folds = TimeSeriesSplit(10)

preds = np.zeros(len(test2))
scores = []

for fold, (idx_train, idx_valid) in enumerate(folds.split(X, y)):
    X_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
    X_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]
    
    xgb = XGBRegressor(**xgb_params, 
          predictor='gpu_predictor', 
          gpu_id=0)

    xgb.fit(
        X_train, 
        y_train,
        eval_metric='mae',
        early_stopping_rounds=100,
        eval_set=[(X_valid, y_valid)], 
        verbose=False
        )
    
    pred_valid = xgb.predict(X_valid)#[:,1]
    #fpr, tpr, _ = roc_curve(y_valid, pred_valid)
    score = smape(y_valid, pred_valid)
    scores.append(score)
    
    print(f"Fold: {fold + 1} Score: {score}")
    print('||'*40)
    
    #test_preds = xgb.predict(test)#[:,1]
    #preds.append(test_preds)
    
    preds += xgb.predict(test2) / folds.n_splits
    
print(f"Overall Validation Score: {np.mean(scores)}")


# Final output

In [ ]:
sample_fp = '../input/tabular-playground-series-mar-2022/sample_submission.csv'
sample = pd.read_csv(sample_fp)

In [ ]:
#preds = xgb.predict(test2)

In [ ]:
preds

In [ ]:
submission = sample.copy()

submission['congestion'] = preds.round(decimals=0)

submission

In [ ]:
submission.to_csv('submission.csv', index=False)